## Exercise 5

In [17]:
import numpy as np
import pandas as pd

Firstly we read in the data X and Y and transform it a bit. Since we also have a $\beta_0$ that works as intercept, we need to add a column with 1's to the matrix $X$. 

In [18]:
with open('X.txt', 'r') as handle:
    X = handle.readlines()
X = [i.split('\n')[0] for i in X]
X = [i.split(',') for i in X]
# the following could prob. be done differently, 
# but i thought it was easier that way
X_df = pd.DataFrame(X)
X_df[0] = X_df[0].astype(float)
X_df[1] = X_df[1].astype(float)
X_df[2] = X_df[2].astype(float)
X_df = X_df.rename(columns = {0:'X1', 1:'X2', 2:'X3'})
# addition of the interception beta_0, as a column of 1 in the Matrix X
X_df['X0'] = 1
# reordering, to have the intercept as first column
X_df = X_df[['X0', 'X1', 'X2', 'X3' ]]
X = X_df.values

with open('Y.txt', 'r') as handle:
    Y = handle.readlines()
Y = [i.split('\n')[0] for i in Y]
Y = [float(i) for i in Y]
Y_df = pd.DataFrame(Y)
Y_df = Y_df.rename(columns = {0:'Y'})
Y = Y_df.values


Now we calculate $\hat{\beta} = (X^TX)^{-1}X^Ty$

In [19]:
beta_hat = np.dot(np.linalg.inv(np.dot(X.T , X)) , np.dot(X.T , Y))
print(beta_hat)

[[-0.00800698]
 [ 0.88161162]
 [-2.45938171]
 [-0.97715699]]


So our calculated $\beta$ values are:

$\hat{\beta_0} = -0.00800698$ , $\hat{\beta_1} = 0.88161162$ , $\hat{\beta_2} = -2.45938171$ , $\hat{\beta_3} = -0.97715699$ 

Having these estimators, we can have a look at the observations (X1, Y), (X2, Y), (X3, Y) and get a sense of wether these linear coefficients give a good fit.


### Estimation of $\widehat{\sigma^2}$ using $\widehat{\beta}$

Now we want to estimate $\widehat{\sigma^2}$ using $\widehat{\beta}$, which we calculated in the lectures as 
$\begin{align}
    \sigma^2 &= \frac{1}{n} \epsilon^T \epsilon \\
    \sigma^2_{ad} &= \frac{1}{n-p-1} \epsilon^T \epsilon 
\end{align}$
For the residuals $\epsilon$, we have $\epsilon = y - X\hat{\beta}$

In [20]:
residuals = Y - np.dot(X, beta_hat)

In [21]:
n = 201 # or n = len(Y)
sigma_sq_hat = 1/n * np.dot(residuals.T, residuals)
print(sigma_sq_hat)

[[0.95484056]]


So the we have $\widehat{\sigma^2} = 0.95484056$.

In [22]:
p = 3
sigma_sq_hat_adj = 1/(n - p - 1) * np.dot(residuals.T, residuals)
print(sigma_sq_hat_adj)


[[0.97422819]]


and $\widehat{\sigma^2}_{ad} = 0.97422819$.

Here we can also see, that the estimator $\widehat{\sigma^2}$ underestimates the true value of $\sigma^2$ and that the adjusted estimator $\widehat{\sigma^2_{ad}}$  is larger and also closer to the true value of $\sigma^2$

# Exercise 8 Problem 3 

Calculating the Ridge regression, using the same data set and comparing the results

Now we calculate $\hat{\beta} = (X^TX + \lambda I_p)^{-1}X^Ty$

In [23]:
beta_hat_ridge = {
    1: np.nan,
    5: np.nan,
    20: np.nan,
    100: np.nan,
    500: np.nan
}
lambda_mapping = {
    1: 0,
    5: 1,
    20: 2,
    100: 3,
    500: 4
}
for l in beta_hat_ridge.keys():
    beta_hat_ridge[l] = np.dot(np.linalg.inv(np.dot(X.T , X) + np.dot(l,np.identity(p+1))) , np.dot(X.T , Y))
print('Linear regression:')
print(beta_hat.flatten())

print('Ridge regression:')
for k,v in beta_hat_ridge.items():
    print(f'lambda: {k}')
    print(f'estimates betas: ')
    print(v.flatten())

Linear regression:
[-0.00800698  0.88161162 -2.45938171 -0.97715699]
Ridge regression:
lambda: 1
estimates betas: 
[-0.00661661  0.87170097 -2.43608212 -0.97613668]
lambda: 5
estimates betas: 
[-1.43268680e-03  8.34103365e-01 -2.34715043e+00 -9.72158696e-01]
lambda: 20
estimates betas: 
[ 0.01363349  0.71705655 -2.06465393 -0.95846534]
lambda: 100
estimates betas: 
[ 0.0413792   0.402319   -1.25834761 -0.90237817]
lambda: 500
estimates betas: 
[ 0.03165688  0.10999069 -0.42691886 -0.7275989 ]


## Analysis:


In [24]:
residuals = Y - np.dot(X, beta_hat)
n = 201 # or n = len(Y)
sigma_sq_hat = 1/n * np.dot(residuals.T, residuals)
p = 3
sigma_sq_hat_adj = 1/(n - p - 1) * np.dot(residuals.T, residuals)
print('Standard linear regression:')
print(sigma_sq_hat)
print(sigma_sq_hat_adj)
for l in beta_hat_ridge.keys():
    print(f'Ridge regression: lambda = {l}')
    residuals = Y - np.dot(X, beta_hat_ridge[l])
    n = 201 # or n = len(Y)
    sigma_sq_hat = 1/n * np.dot(residuals.T, residuals)
    p = 3
    sigma_sq_hat_adj = 1/(n - p - 1) * np.dot(residuals.T, residuals)
    print(f'sigma squared: {sigma_sq_hat}')
    print(f'adjusted sigma squared: {sigma_sq_hat_adj}')

Standard linear regression:
[[0.95484056]]
[[0.97422819]]
Ridge regression: lambda = 1
sigma squared: [[0.95517093]]
adjusted sigma squared: [[0.97456526]]
Ridge regression: lambda = 5
sigma squared: [[0.96250024]]
adjusted sigma squared: [[0.98204339]]
Ridge regression: lambda = 20
sigma squared: [[1.04942681]]
adjusted sigma squared: [[1.07073497]]
Ridge regression: lambda = 100
sigma squared: [[1.83522968]]
adjusted sigma squared: [[1.87249322]]
Ridge regression: lambda = 500
sigma squared: [[3.77297588]]
adjusted sigma squared: [[3.84958453]]
